# Download Yield Data from USDA NASS

Date: 09/01/2022

Note: Please install NASS data api, nass, 
with code `!pip install nass` before importing the packages
if you haven't yet.

Please request an api key at https://quickstats.nass.usda.gov/api/ and insert in the first line of the function `get_data_USDA` below.

In [ ]:
import pandas as pd
import nass
import os

### Download available corn yield data for all counties from 2000 to 2020

In [ ]:
# Import cencus data for county ID.
cwd = os.path.dirname(os.getcwd())
filename = 'Data_original\\All_counties.xlsx'
location = os.path.join(cwd, filename)
All_counties_df = pd.read_excel(location, engine='openpyxl', header=0)

In [ ]:
# Clean the county dataset
All_counties_df.Areaname = All_counties_df.Areaname.astype(str)
All_counties_df['county_name'] = [x[:-4].upper() 
                                  for x in All_counties_df['Areaname']]
All_counties_df['state_abr'] = [x[-2:].upper() 
                                for x in All_counties_df['Areaname']]
All_counties_df['state_code'] = ['{0:02}'.format(x//1000) 
                                 for x in All_counties_df['STCOU']]
All_counties_df['county_code'] = ['{0:03}'.format(x - 1000*(x//1000)) 
                                  for x in All_counties_df['STCOU']]
All_counties_df = All_counties_df[All_counties_df['county_code']!='000']
All_counties_df.county_code = All_counties_df.county_code.astype(str)
All_counties_df.state_code = All_counties_df.state_code.astype(str)
All_counties_df.reset_index(drop=True, inplace=True)

In [ ]:
def get_data_USDA(category):
    # Please request an api key at https://quickstats.nass.usda.gov/api/.
    # Please input your api key here.
    api = nass.NassApi()# Please input your API here.
    count = 0
    for i in range(len(All_counties_df)):
        state_code = All_counties_df.loc[i, 'state_code']
        county_code = All_counties_df.loc[i, 'county_code']
        q = api.query()
        q.filter('state_fips_code', state_code)
        q.filter('county_code', county_code)
        q.filter('short_desc', category)
        try:
            df = pd.DataFrame(q.execute())
            if count == 0:
                output_df = df
            else:
                output_df = pd.concat([output_df, df], ignore_index=True)
            count += 1
        except Exception:
            print(All_counties_df.loc[i, 'Areaname'])
            pass
    
    return output_df

In [ ]:
corn_yield_data = get_data_USDA('CORN, GRAIN - YIELD, MEASURED IN BU / ACRE')

In [ ]:
filename = 'Data_original\\corn_yield_data.csv'
location = os.path.join(cwd, filename)
corn_yield_data.to_csv(location)